In [1]:
import win32com.client
import pandas as pd
import shutil
import os
import subprocess
from datetime import datetime
import folium
from geopy.geocoders import Photon
from html2image import Html2Image
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import unicodedata
import math
import warnings
import time
import pytz
import re
#from win32com.client import gencache
#gencache._Dump()


warnings.filterwarnings('ignore')

kpmg_dark_blue = 51*256+141*256*256
kpmg_white = 255+255*256+255*256*256

class dashboard_tprs:
    def __init__(self,aml_files_path,template_path ,working_dir=os.getcwd(),send_to="bekert@kpmg.pl"):
        self.working_dir = working_dir
        self.aml_files_path = aml_files_path
        self.risk_score_path = os.path.join(aml_files_path,"RiskScore.xlsx")
        self.aml_path = os.path.join(aml_files_path,"AMLdata.xlsx")
        self.finial_pdf = os.path.join(working_dir,"TPRS_final.pdf")
        self.template_path = template_path
        self.mail_to = send_to
        os.chdir(working_dir)
        

    def prepare_risk_score(self):
        df_risk_score = pd.read_excel(self.risk_score_path,sheet_name="RiskScore",na_filter=False)
        df_risk_score['RiskPoints']=df_risk_score['RiskPoints'].astype('int')
        df_risk_sum = df_risk_score.groupby(["NIP_detail","Kategoria"])['RiskPoints'].sum().unstack(level=-1).fillna(0).reset_index()
        df_risk_sum["Total"] = df_risk_sum['Ryzyko finansowe']+df_risk_sum['Ryzyko biznesowe']
        df_risk_sum['NIP_detail'] = df_risk_sum['NIP_detail'].astype('str')
        df_risk_sum['Ryzyko biznesowe']=df_risk_sum['Ryzyko biznesowe'].astype('int')
        df_risk_sum['Ryzyko finansowe']=df_risk_sum['Ryzyko finansowe'].astype('int')
        self.df_risk_sum = df_risk_sum

    def prepare_address(self):
        df_adres = pd.read_excel(self.aml_path,sheet_name="Adres",na_filter=False)
        df_adres =df_adres.loc[(df_adres['A/H']=="A") &(df_adres['Address_type']=='adres_dzialalnosci')]
        #df_adres['NIP_base'] = df_adres['NIP_base'].astype('str')
        for col in df_adres.columns:
            df_adres[col] = df_adres[col].astype('str')
        df_adres['adres_to_loc'] = df_adres['Budynek'].str.cat(others=df_adres[['Street','City','PostalCode','Country']],sep=" , ",na_rep="")
        #df_adres['adres_to_loc'] = df_adres['adres_to_loc'].apply(lambda x: 
           #                                                       unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode())
        self.df_adres = df_adres

    def prepare_edge(self):
        df_edge = pd.read_excel(self.aml_path,sheet_name="Edge",na_filter=False)
        df_edge_ac = df_edge.loc[ (df_edge['A/H']=='A')]
        base_df = df_edge_ac[['NIP_badany','NIP_detail','id','ShortName','Funkcja_new']]
        base_df['NIP_detail'] = base_df['NIP_detail'].astype('str')
        base_df['NIP_badany'] = base_df['NIP_badany'].astype('str')
        
        df_edge.loc[df_edge['LP_order']=="",'LP_order'] = 999999
        df_edge['LP_order'] = df_edge['LP_order']*100
        df_edge_id = df_edge.loc[(df_edge['C/R']=='C')].groupby(['NIP_detail','Funkcja_new','A/H'])['LP_order'].min().reset_index()
        df_edge_id['LP_order'] = df_edge_id['LP_order']-1
        df_edge_wor = pd.concat([df_edge,df_edge_id],ignore_index=True).fillna("")
        df_edge_wor.rename(columns={"ShortName":"Funkcja","Concat_Od_Do":"Od"},inplace=True)
        df_edge_wor["NIP_detail"] = df_edge_wor["NIP_detail"].astype("str")
        self.df_edge_wor_A = df_edge_wor.loc[(df_edge_wor['A/H']=='A')].sort_values(['LP_order'])
        self.df_edge_wor_H = df_edge_wor.loc[(df_edge_wor['A/H']=='H')].sort_values(['LP_order'])

        base_df['ShortName'] = base_df['ShortName'].str.replace(" SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ","")
        edges_fin = base_df.merge(base_df[['NIP_badany','id','ShortName']].drop_duplicates(),how='left',
                      left_on=['NIP_badany', 'NIP_detail'],right_on=['NIP_badany','id'])
        self.edges_fin_clen = edges_fin.loc[~edges_fin['ShortName_y'].isna()]
        

    def create_risk_plot(self,r_b,r_f):
        if r_b<14:
            c_r_b = "green"
        elif r_b<30:
            c_r_b = "yellow"
        else:
            c_r_b="red"
        
        if r_f<14:
            r_f_b = "green"
        elif r_f<30:
            r_f_b = "yellow"
        else:
            r_f_b="red"
        image = "risk_plot.png"
        bar_plt = sns.barplot(x=[r_b, r_f],y=[f'{r_b}\nRyzyko biznesowe',f'{r_f}\nRyzyko finansowe'],orient='h',palette=[c_r_b,r_f_b],
                             width=0.4)
        plt.box(False)
        bar_plt.set(xticklabels=[])
        bar_plt.tick_params(bottom=False,left=False)
        bar_plt.margins(y=5)
        bar_plt.axvline(x=14, color='black', linestyle='-')
        bar_plt.axvline(x=30, color='black', linestyle='-')
        bar_plt.set_ylim(-0.5,1.5)
        sns.set(font_scale=2)
        bar_plt.figure.savefig(image,bbox_inches='tight')
        plt.clf()
        return os.path.join(self.working_dir,image)
    
    def create_spider(self, nodes, filtered):
        plt.rcParams["figure.figsize"] = (20, 14)
        image = 'pajak.png'
        
        edge_dcit = {}
        for x,y,fun in filtered.groupby(['id_x','id_y'])['Funkcja_new'].apply(lambda x: "\n".join(x)).reset_index().values:
            edge_dcit[(x,y)]=fun
        
        # Create an empty graph and add nodes
        G = nx.Graph()
        label_dict = {}
        for label, nip in nodes.values:
            G.add_node(nip)
            label_dict[nip] = label
            
        # Add edges between nodes to create connections
        for x,y in filtered[['id_x','id_y']].values:
            G.add_edge(x,y)
        # Create a dictionary of the edge labels
        pos_changed={}
        pos = nx.spring_layout(G)
        for k,v in pos.items():
            pos_changed[k] =[v[0]*50,v[1]*50]
        
        nx.draw_networkx_nodes(G, pos_changed,node_size=0)
        nx.draw_networkx_edges(G, pos_changed, alpha=0.4,arrows=True,min_source_margin =25, min_target_margin=25)
        nx.draw_networkx_labels(G, pos_changed,font_size=7,labels=label_dict)
        
        nx.draw_networkx_edge_labels(G, pos_changed, edge_labels=edge_dcit, label_pos=0.5, font_size=7, font_color='black',alpha=0.5)
        plt.axis('off')
        plt.tight_layout()
        #plt.margins(x=1)
        axis = plt.gca()
        axis.set_xlim([1.2*x for x in axis.get_xlim()])
        #axis.set_ylim([0.4*y for y in axis.get_ylim()])
        plt.savefig(image,bbox_inches="tight")
        #plt.clf()
        return os.path.join(self.working_dir,image)
    
    def create_map(self,address):
        geolocator = Photon(user_agent="tprs")
        location = geolocator.geocode(address)
        lat = location.latitude
        lng = location.longitude
        map = folium.Map(location=[location.latitude,location.longitude],zoom_start=11)
        folium.Marker([location.latitude,location.longitude],tooltip="name").add_to(map)
        map.save("map.html")
        hti = Html2Image(browser='edge',custom_flags=['--virtual-time-budget=200'],output_path=self.working_dir)
        image = "mapa.png"
        finial_path = os.path.join(self.working_dir,image)
        with open(os.path.join(self.working_dir,"map.html")) as f:
            str_ = f.read()
            hti.screenshot(str_, save_as=image,size=(600,400))
        return finial_path
        
    def insert_image(self,sheet,path_to_image,row,col,width=200):
        pic = sheet.Pictures().Insert(path_to_image)
        pic.Left = sheet.Cells(row,col).Left  
        pic.Top = sheet.Cells(row,col).Top
        pic.ShapeRange.LockAspectRatio =True
        pic.width=width
        
    def insert_table(self,worksheet,df_tab,cols_to_sel,cols_final,row_,col_,highlight_col="",check_col="" ,pg_break=92):
        first_pg = pg_break-row_
        pages = math.ceil((df_tab.shape[0]-first_pg)/(pg_break-4))+1
        df_fin = df_tab[cols_to_sel]
        cols_range = len(cols_final)
        row_nr =1
        for page in range(pages):
            if page==0:
                start_row = 0
                end_row = first_pg
            else:
                start_row = end_row
                end_row = start_row+pg_break
                row_ = pg_break*page+(4*page)
                worksheet.HPageBreaks.Add(Before=worksheet.Rows(row_-1))
            print(start_row,end_row,row_)
            for i,col in enumerate(cols_final):
                temp_row = row_-1+row_nr
                temp_col = col_+i
                worksheet.Cells(temp_row,temp_col).Value = col
                worksheet.Cells(temp_row,temp_col).Interior.Color =kpmg_dark_blue
                worksheet.Cells(temp_row,temp_col).Font.Color =kpmg_white
                worksheet.Cells(temp_row,temp_col).WrapText =True
            for _, row in df_fin.iloc[start_row:end_row].iterrows():
                if highlight_col:
                    if row[check_col] == "":
                        row[check_col] = row[highlight_col]
                        highL = True
                        #print(row[check_col])
                for col_nr,col in enumerate(cols_final):
                    temp_row = row_+row_nr
                    temp_col = col_+col_nr
                    
                    if highL:
                        worksheet.Cells(temp_row,temp_col).Font.Bold = True
                    if (not highL) & (col_nr==0) & (highlight_col!=""):
                        indi=4
                    else:
                        indi=0
                    #print(temp_row,temp_col)
                    worksheet.Cells(temp_row,temp_col).Value = f"{indi*' '}{row[col]}"
                    worksheet.Cells(temp_row,temp_col).WrapText =True
                    if highL:
                        worksheet.Cells(temp_row,temp_col).Font.Bold = True
                row_nr =row_nr+1
                highL = False
            row_nr = 0
    
    def save_excel_as_pdf(self):
        excel = win32com.client.DispatchEx("Excel.Application")
        excel.DisplayAlerts =False
        wb = excel.Workbooks.Open(self.template_path)
        wb.AutoSaveOn=False
        source_sheet = wb.Sheets("template")

        df_key = pd.read_excel(self.aml_path,sheet_name="KluczoweDane",na_filter=False).iloc[:3]
        for n,row in df_key.iterrows():
            nip = str(row['NIP_base'])
            adres = self.df_adres.loc[self.df_adres['NIP_base']==nip,'adres_to_loc'].iloc[0]+" , Poland"
            source_sheet.Copy(None, wb.Sheets(wb.Sheets.Count))
            new_sheet = wb.Sheets(wb.Sheets.Count)
            new_sheet.Name = str(n)
            
            #Wrzucenie danych podstawowych
            new_sheet.Range("D6").Value = row['Nazwa_skr']
            new_sheet.Range("D8").Value = row['RozpoczeciaDzialalnosci']
            new_sheet.Range("D10").Value = row['DataRejestKRS']
            new_sheet.Range("D12").Value = nip
            new_sheet.Range("D14").Value = row['REGON']
            new_sheet.Range("D16").Value = row['KRS']
            new_sheet.Range("D18").Value = row['Status']
            new_sheet.Range("D20").Value = row['kapital']
            
            #Wykres ryzyka
            r_b = self.df_risk_sum.loc[self.df_risk_sum['NIP_detail']==nip,'Ryzyko biznesowe'].iloc[0]
            r_f = self.df_risk_sum.loc[self.df_risk_sum['NIP_detail']==nip,'Ryzyko finansowe'].iloc[0]
            new_sheet.Range("I10").Value = r_f
            new_sheet.Range("I8").Value = r_b
            new_sheet.Range("I6").Value = self.df_risk_sum.loc[self.df_risk_sum['NIP_detail']==nip,'Total'].iloc[0]
            self.create_risk_plot(r_b, r_f)
            
            #try:
            path_to_image = self.create_risk_plot(r_b, r_f)
            self.insert_image(new_sheet, path_to_image,5,10)
            #except Exception as e:
            #print(f"error risk plot {e}")
            
            try:
                path_to_image = self.create_map(adres)
                self.insert_image(new_sheet, path_to_image,15,7)
            except Exception as e:
                print(f"error mapy plot {e}")


            #wrzucenie tabeli edge
            cols_to_sel = ['Funkcja','Funkcja_new','Od','Wiek','l.pow.akt','l.pow.hist']
            cols_final = ['Funkcja','Od','Wiek','l.pow.akt','l.pow.hist']

            df_tab = self.df_edge_wor_A.loc[(self.df_edge_wor_A['NIP_detail']==nip) ]
            #df_tab = self.df_edge_wor_A.iloc[:199]
            self.insert_table(new_sheet,df_tab,cols_to_sel,cols_final,60,3,highlight_col="Funkcja_new",check_col="Funkcja" ,pg_break=90)

            filtered_df = self.edges_fin_clen.loc[self.edges_fin_clen["NIP_badany"]==nip]
            unique_list = []
            final_list = []
            for x,y,fun,id_x,id_y in filtered_df[['ShortName_x','ShortName_y','Funkcja_new','id_x','id_y']].values:
                if set([x,y,fun,id_x,id_y]) not in unique_list:
                    unique_list.append(set([x,y,fun,id_x,id_y]))
                    final_list.append([x,y,fun,id_x,id_y])
            filtered = pd.DataFrame(final_list,columns=['ShortName_x','ShortName_y','Funkcja_new','id_x','id_y'])
            nodes = pd.concat([filtered[['ShortName_x','id_x']] ,filtered[['ShortName_y','id_y']].rename(columns={"ShortName_y":'ShortName_x',"id_y":"id_x"})]).drop_duplicates()
            
            try:
                path_to_image = self.create_spider(nodes, filtered)
                self.insert_image(new_sheet, path_to_image,23,2,600)
            except Exception as e:
                print(f"error mapy plot {e}")

        
        wb.Sheets("template").Delete()
        wb.ExportAsFixedFormat(0,self.finial_pdf, IncludeDocProperties=True,IgnorePrintAreas=False,Quality=0)
        wb.Close(False)
        excel.Quit()
        self.send_mail("Final result","File attached")
    
    def send_mail(self,subject, content):
        self.outlook = win32com.client.Dispatch('outlook.application')
        mail = self.outlook.CreateItem(0)
        mail.To = self.mail_to 
        mail.Subject = subject
        mail.Body = content
        mail.Attachments.Add(self.finial_pdf)
        mail.Send()



class cmd_runner:
    def __init__(self, alteryx_path, mail_to,mail_cc=""):
        self.alteryx_path = alteryx_path
        self.mail_to = mail_to
        self.mail_cc = mail_cc
        self.outlook = win32com.client.Dispatch('outlook.application')
        #outlook = win32com.client.Dispatch("Outlook.Application")
    def run_workflow(self, path_to_workflow):
        code = f'"{self.alteryx_path}" "{path_to_workflow}"'
        proc = subprocess.Popen(code, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        out, err = proc.communicate()
        errors_list = [line for line in str(out).split("\\n") if line.lower().startswith("error")]
        workflow_name = os.path.basename(path_to_workflow)
        body=f"Path to workflow:{path_to_workflow}\n\n"
        if errors_list:
            subject = f"Workflow {workflow_name} contains error"
            body=body+'\n'.join(errors_list)
            self.send_mail(subject, body)
            return 1
        else:
            subject = f"Workflow {workflow_name} runned successfully"
            self.send_mail(subject, body)
            return 0
    def send_mail(self,subject, content,error_mail=""):
        mail = self.outlook.CreateItem(0)
        if error_mail:
            mail.To = error_mail
        else:
            mail.To = self.mail_to 
        mail.Subject = subject
        mail.Body = content
        mail.Send()
class tprs_runner(cmd_runner):
    def __init__(self, nip_list, email_info, working_dir, alteryx_path=r"C:\Users\bekert\AppData\Local\Alteryx\bin\alteryxenginecmd", mail_to="bekert@kpmg.pl",mail_cc=""):
        self.nip_list = nip_list
        self.email_info = email_info
        self.working_dir = working_dir
        super().__init__(alteryx_path, mail_to,mail_cc="")
        #self.working_path = r"C:\Users\bekert\OneDrive - KPMG\Alteryx\7.TPRS\240327_14_39_08"
        #self.working_path = r"C:\Users\bekert\OneDrive - KPMG\Alteryx\7.TPRS\240404_10_38_47" #do usuniecia

    
    def copy_new_folder(self):
        self.time_str = datetime.now().strftime("%y%m%d_%H_%M_%S")
        self.working_path = os.path.join(os.path.dirname(self.working_dir),self.time_str)
        shutil.copytree(self.working_dir,self.working_path)

    def create_nip_file(self,path_nip="3.Alteryx\API_flows\input",tprs="Y"):
        nip_file_path  = os.path.join(self.working_path,path_nip)
        df_nip = pd.DataFrame(self.nip_list,columns=['NIP'])
        if tprs=="Y":
            for name in ["EMIS","CDEIG","CRBR","KRS","MP","VIES","BKV"]:
                df_nip[name]="Y"
        df_nip.to_excel(f"{nip_file_path}/NIP_input.xlsx",index=False)
    def run_workflow(self, path_to_workflow,base_path="3.Alteryx\API_flows"):
        path_to_workflow = os.path.join(self.working_path,base_path,path_to_workflow)
        if super().run_workflow(path_to_workflow):
            return 1

class proposal_runner(tprs_runner):
    def regon_check_avail(self,base_path="1.Alteryx\input"):
        df_regon_check = pd.read_excel(os.path.join(self.working_path,base_path,"regon_avail.xlsx"),na_filter=False,sheet_name='sheet1')
        errors_ =[]
        for val in df_regon_check['ErrorMessagePl']:
            if val!="":
                errors_.append(val)
        return "\n".join(errors_)
            
    def prepare_excel(self):
        self.prop_path = os.path.join(self.working_path,"1.Alteryx\output",f"{self.time_str}_Proposal.xlsx")
        excel = win32com.client.DispatchEx('Excel.Application')
        excel.DisplayAlerts =False
        excel.Visible=False
        df_codes_path =  os.path.join(self.working_path,"1.Alteryx\output\Std_codes.xlsx")
        df_codes = pd.read_excel(df_codes_path,sheet_name="codes",na_filter=False)
        df_codes['Standard_ID'] = df_codes['Standard_ID'].astype('str')
        wb = excel.Workbooks.Open(self.prop_path)
        
        if not df_codes['Standard_ID'].isin(["1141","1142"]).any():
            wb.Worksheets('BS').Delete()
        if not df_codes['Standard_ID'].isin(["1141"]).any():
            wb.Worksheets('P&L_calc').Delete()
        if not df_codes['Standard_ID'].isin(["1142"]).any():
            wb.Worksheets('P&L_comp').Delete()  
        
        if not df_codes['Standard_ID'].isin(["1231"]).any():
            wb.Worksheets('BS_micro').Delete()
            wb.Worksheets('P&L_micro').Delete()
        if not df_codes['Standard_ID'].isin(["1162"]).any():
            wb.Worksheets('BS_stock').Delete()
            wb.Worksheets('P&L_stock').Delete()
            
        for sheet in wb.Worksheets:
            if sheet.AutoFilterMode:
                sheet.Range("B5").AutoFilter(Field=1, Criteria1="<>Hide")
                sheet.Rows(6).Hidden = True
        wb.Close(True)
        time.sleep(18)
        excel.Quit()
        
    def send_final_mail(self,subject,content,finial_recivier=""):
        self.outlook = win32com.client.Dispatch('outlook.application')
        mail = self.outlook.CreateItem(0)
        if finial_recivier:
            mail.To = finial_recivier
        else:
            mail.To = self.mail_to 
        mail.Subject = subject
        mail.Body = content
        mail.Attachments.Add(self.prop_path)
        mail.Send()

class run_comp_10(tprs_runner):
    def send_final_mail(self,subject,content,finial_recivier="",attch=""):
        self.outlook = win32com.client.Dispatch('outlook.application')
        mail = self.outlook.CreateItem(0)
        if finial_recivier:
            mail.To = finial_recivier
        else:
            mail.To = self.mail_to 
        mail.Subject = subject
        mail.Body = content
        if attch:
            mail.Attachments.Add(attch)
        mail.Send()
    def save_attch(self,mail,path="1.Alteryx\input"):
        for att in email.Attachments:
            if "xlsx" in  att.FileName: 
                att.SaveAsFile(os.path.join(self.working_path,path,"NIP.xlsx"))

In [2]:

def run_tprs(email,nip_list):
    alterx_path = r"C:\Users\bekert\AppData\Local\Alteryx\bin\alteryxenginecmd"
    mail_to ="bekert@kpmg.pl"
    working_dir = r"C:\Users\bekert\OneDrive - KPMG\Alteryx\7.TPRS\00. TPRS_PustyDoKopiowania"
    template_path = r"C:\Users\bekert\OneDrive - KPMG\Documents\python\tprs\Template.xlsx"
    working_path_dashboard = r"C:\Users\bekert\OneDrive - KPMG\Documents\python\tprs\Working_dir"
    email_info="test"
    
    if email.SenderEmailType =="EX":
        sender = email.Sender.GetExchangeUser().PrimarySmtpAddress
    else:
        sender = email.SenderEmailAddress
    #"0.2 Dlugi_scrappingy_v01.yxmd"
    workflow_list = ["0.1 API_Tree_Runner_v3.yxmd","0.2 Dlugi_scrappingy_v01_short.yxmd","1. AML Data v3.7be.yxmd","2. AML screening v2.4kw.yxmd","3. Finance Data v2.29.yxmd","4. AML risk points v2.9kw.yxmd"]
    #workflow_list = ["1. AML Data v3.7be.yxmd","2. AML screening v2.4kw.yxmd","3. Finance Data v2.29.yxmd","4. AML risk points v2.9kw.yxmd"]
    #workflow_list = ["4. AML risk points v2.9kw.yxmd"]
    test_run = tprs_runner(nip_list, email_info, working_dir)
    test_run.copy_new_folder() #do odkomentowania
    test_run.create_nip_file()
    for workflow in workflow_list:
        if test_run.run_workflow(workflow):
            print(F"Error {workflow}")
            test_run.send_mail("Issue in Automation","Apology, our automatization identified issue, \n our team has been noticed and currently is working on solution",sender)
            return 1
    
    aml_path = os.path.join(test_run.working_path,"3.Alteryx\API_flows\output")
    dashboard_creation = dashboard_tprs(aml_path,template_path,working_path_dashboard, sender)
    dashboard_creation.prepare_risk_score()
    dashboard_creation.prepare_address()
    dashboard_creation.prepare_edge()
    dashboard_creation.save_excel_as_pdf()



def run_prop(email,nip_list):
    working_dir = r"C:\Users\bekert\OneDrive - KPMG\Alteryx\9.ProposalBooster\00.PustyDoKopiowania"
    email_info = "Test Proposal"
    workflow_list_prop = ["API_project_v14.yxmd", "ProposalBooster_v14.yxmd"]
    
    if email.SenderEmailType =="EX":
        sender = email.Sender.GetExchangeUser().PrimarySmtpAddress
    else:
        sender = email.SenderEmailAddress
        
    prop_runner = proposal_runner(nip_list, email_info, working_dir)
    prop_runner.copy_new_folder() #do odkomentowania
    prop_runner.create_nip_file("1.Alteryx\input","N")

    prop_runner.run_workflow("regon_avability_check.yxmd","1.Alteryx")
    erros_regon = prop_runner.regon_check_avail()
    if erros_regon:
        prop_runner.send_mail("Nip Issue",erros_regon,sender)
        return
    for workflow in workflow_list_prop:
        if prop_runner.run_workflow(workflow,"1.Alteryx"):
            print(F"Error {workflow}")
            prop_runner.send_mail("Issue in Automation","Apology, our automatization identified issue, \n our team has been noticed and currently is working on solution",sender)
            return 1
            
    prop_runner.prepare_excel()
    prop_runner.send_final_mail("Final prop booster","File attached",sender)


def run_10(email):
    working_dir = r"C:\Users\bekert\OneDrive - KPMG\Alteryx\22.Benchmarkng Updates\5. Benchmarks\0. PustyDoKopiowania"
    workflow_name = "1. Benchmarking_10.yxmd"
    if email.SenderEmailType =="EX":
        sender = email.Sender.GetExchangeUser().PrimarySmtpAddress
    else:
        sender = email.SenderEmailAddress
    Comp_10 = run_comp_10("", "", working_dir)
    Comp_10.copy_new_folder() #do odkomentowania
    Comp_10.save_attch(email)
    Comp_10.run_workflow(workflow_name,"1.Alteryx")
    final_file = os.path.join(Comp_10.working_path,"1.Alteryx\output","Benchmarking_10.xlsx")
    Comp_10.send_final_mail("Final Comp_10","File attached",sender,final_file)

# connect to Outlook
outlook = win32com.client.Dispatch('outlook.application').GetNamespace("MAPI")

utc=pytz.UTC
root_folder = outlook.Folders["bekert@kpmg.pl"].Folders["Inbox"]
done_cor = outlook.Folders["bekert@kpmg.pl"].Folders["Inbox"].Folders["Done_cor"]
issue_path = outlook.Folders["bekert@kpmg.pl"].Folders["Inbox"].Folders["Issue"]

items = root_folder.Items
items.Sort('[ReceivedTime]', True)
start_auto_date = utc.localize(datetime.now())
subjects_checked = []
subjects_checked_prop = []
to_check_list = []
to_check_list_prop = []
to_check_run_10 = []
while True:
    for email in items:
        #if (email.ReceivedTime >= start_auto_date ) & ("tprs" in email.Subject.lower()) & (email.Subject not in subjects_checked):
        if (email.ReceivedTime >= start_auto_date ) & ("tprs" in email.Subject.lower()):
            subjects_checked.append(email.Subject)
            to_check_list.append([email, re.findall('\d{10}', email.Body)])
        #if (email.ReceivedTime >= start_auto_date ) & ("proposal booster" in email.Subject.lower()) & (email.Subject not in subjects_checked_prop):
        if (email.ReceivedTime >= start_auto_date ) & ("proposal booster" in email.Subject.lower()):   
            subjects_checked_prop.append(email.Subject)
            to_check_list_prop.append([email, re.findall('\d{10}', email.Body)])
        if (email.ReceivedTime >= start_auto_date ) & ("bench 10" in email.Subject.lower()):   
            to_check_run_10.append(email)
    #start_auto_date = utc.localize(datetime.now())
    if to_check_list:
        for email, nip_list in to_check_list:
            try:
                run_tprs(email, nip_list)
                email.Move(done_cor)
            except Exception as e:
                print(e)
                email.Move(issue_path)
    if to_check_list_prop:
        for email, nip_list in to_check_list_prop:
            try:
                run_prop(email, nip_list)
                email.Move(done_cor)
            except Exception as e:
                print(e)
                email.Move(issue_path)
    if to_check_run_10:
        for email in to_check_run_10:
            try:
                run_10(email)
                email.Move(done_cor)
            except Exception as e:
                print(e)
                email.Move(issue_path)
    time.sleep(2)
    to_check_list = []
    to_check_list_prop=[]
    to_check_run_10=[]


KeyboardInterrupt



In [31]:
working_dir = r"C:\Users\bekert\OneDrive - KPMG\Alteryx\9.ProposalBooster\00.PustyDoKopiowania"
nip_list =["1182180817"]
email_info = "Test Proposal"
prop_runner = proposal_runner(nip_list, email_info, working_dir)
prop_runner.copy_new_folder() #do odkomentowania
prop_runner.create_nip_file("1.Alteryx\input","N")

In [32]:
workflow_list_prop = ["API_project_v13.yxmd", "ProposalBooster_v13.yxmd"]
for workflow in workflow_list_prop:
    prop_runner.run_workflow(workflow,"1.Alteryx")

In [33]:
prop_runner.prepare_excel()
prop_runner.send_final_mail("Final prop booster","File attached")

In [34]:
"proposal booster" in "twojastaraproposal booster"

True

In [68]:
df_codes_path = r"C:\Users\bekert\OneDrive - KPMG\Alteryx\9.ProposalBooster\240325_BE_tests\1.Alteryx\output\Std_codes.xlsx"
df_codes = pd.read_excel(test_df_path,sheet_name="codes",na_filter=False)
df_codes['Standard_ID'] = df_codes['Standard_ID'].astype('str')
df_codes['Standard_ID'].isin(["1141"]).any()

In [74]:
df_codes['Standard_ID'].isin(["1141"]).any()

0    True
Name: Standard_ID, dtype: bool

In [115]:
def testttt(x):
    for i in x:
        if i==2:
            return
        else:
            print(i)

In [116]:
testttt([1,56,2,4,5])

1
56


In [11]:
alteryx_path=r"C:\Users\bekert\AppData\Local\Alteryx\bin\alteryxenginecmd"
path_to_workflow=r"C:\Users\bekert\OneDrive - KPMG\Alteryx\22.Benchmarkng Updates\5. Benchmarks\240416_11_37_27\1.Alteryx\1. Benchmarking_10.yxmd"
code = f'"{alteryx_path}" "{path_to_workflow}"'
proc = subprocess.Popen(code, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
out, err = proc.communicate()
errors_list = [line for line in str(out).split("\\n") if line.lower().startswith("error")]

In [12]:
code

'"C:\\Users\\bekert\\AppData\\Local\\Alteryx\\bin\\alteryxenginecmd" "C:\\Users\\bekert\\OneDrive - KPMG\\Alteryx\\22.Benchmarkng Updates\\5. Benchmarks\\240416_11_37_27\\1.Alteryx\\1. Benchmarking_10.yxmd"'

In [13]:
errors_list

[]